# 轉向量

In [ ]:
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd 
from tqdm import tqdm
import numpy as np

# 使用 cuda，如果有 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 載入 BERT 模型和斷詞機制
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)

# 設置模型為評估模式，以防止梯度計算
model.eval()

def sentences2list(path,key_1, key_2):
    df = pd.read_csv(path)
    sentences_1 = df[key_1].values.tolist()
    sentences_2 = df[key_2].values.tolist()

    return sentences_1,sentences_2



def get_sentence_vector(sentence):
    # Tokenize 句子並轉換為張量
    # turning 會引用到
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

    # 用於不進行梯度計算的上下文管理器
    with torch.no_grad():
        # 獲取 BERT 模型的最後一層的輸出
        outputs = model(**inputs)
    
    # 最後一層的所有 token 的向量
    last_hidden_state = outputs.last_hidden_state

    # 將所有 token 的向量相加，並除以 token 的數量，得到句子的向量
    sentence_vector = (last_hidden_state.sum(dim=1) / last_hidden_state.shape[1]).cpu().numpy().tolist()

    return sentence_vector



def turning(sentences_list):
    vectors = []
    total = len(sentences_list)
    for num, sen in tqdm(enumerate(sentences_list),total = total, desc="sentence to vector ..."):
        vector = str(get_sentence_vector(sen))
        vectors.append(vector)
    return vectors

path_=""
sentence_1, sentence_2 = sentences2list(path="", "sen1", "sen2")
vector_1 = turning(sentence_1)
vector_2 = turning(sentence_2)

pd.DataFrame({"sen1":sentence_1,"vector1":vector_1,"sen2":sentence_2,"vector2":vector_2}).to_csv(path_,encoding="utf-8-sig")


# 移除前面檔案的左右括弧

In [ ]:

import pandas as pd 
csv_file = path_

df = pd.read_csv(csv_file)

sen1 = df['sen1'].values.tolist()
vector_1 = df['vector1'].values.tolist()
sen2 = df['sen2'].values.tolist()
vector_2 = df['vector2'].values.tolist()
label = df['label'].values.tolist()

for num, vectors in enumerate(vector_1):
    vector_1[num] = vectors.replace('[[','[').replace(']]',']')

for num, vectors in enumerate(vector_2):
    vector_2[num] = vector_2.replace('[[','[').replace(']]',']')

pd.DataFrame({"sen1":sentence_1,"vector1":vector_1,"sen2":sentence_2,"vector2":vector_2,"label":label}).to_csv(csv_file,encoding='utf-8-sig')

# 推論

In [ ]:
import torch
import pandas as pd
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np 

# 定義輸出 CSV 檔案路徑
input_csv_path = r"C:\Users\YuCheng_Ch\Desktop\1731rand_ans_que_merge_tensor.csv" # 替換成你的路徑
model_path = r"C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train\model_epoch_25_acc_83.49_loss_0.4775.pth"
csv_output_path = r'C:\Users\YuCheng_Ch\Desktop\datasets_accton\預測結果\1731rand_ans_que_merge_tensor-predict.csv'
batch_size = 128



# 定義模型架構
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        


        self.linear1 = nn.Linear(768, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 128)        
        self.linear4 = nn.Linear(128, 64)
        self.linear5 = nn.Linear(64, 16)
        self.linear6 = nn.Linear(16, 8)
        self.output = nn.Linear(8, 2)   

        
    def forward(self, x):
        x = self.linear1(x)
        #print(f'After linear1: {x.shape}')  # 調試輸出
        x = F.relu(x)
        #print(f'After relu1: {x.shape}')  # 調試輸出
        x = self.linear2(x)
        #print(f'After linear2: {x.shape}')  # 調試輸出
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        x = F.relu(x)
        x = self.linear5(x)
        x = F.relu(x)
        x = self.linear6(x)
        x = F.relu(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)
        #print(f'Final output: {x.shape}')  # 調試輸出
  
        return x



df = pd.read_csv(input_csv_path)

# 提取句子和向量
sen = (df['sen1']+df['sen2']).values.tolist()
final_vector = df['final_vector'].apply(eval).tolist()


model = NN()
model.load_state_dict(torch.load(model_path, weights_only=True))
model = model.cuda()
model.eval()

# 轉換向量為Tensor
merged_vectors = torch.tensor(final_vector, dtype=torch.float32).cuda()
max_num = torch.max(merged_vectors)
merged_vectors = merged_vectors/max_num

# 使用 DataLoader 進行批量預測
dataset = TensorDataset(merged_vectors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

class_1_probabilities = []

with torch.no_grad():
    for batch in dataloader:
        batch = batch[0].cuda()
        logits = model(batch)
        print(f'Logits: {logits[:2]}')  # 打印前5個 logits
        logits = F.softmax(logits, dim=1)
        print(f'Softmax: {logits[:2]}')  # 打印前5個 softmax 結果
        class_1_probabilities.extend(logits[:, 1].cpu().numpy())



# 儲存結果到CSV
pd.DataFrame({'sentence': sen, 'class_1_probabilities': class_1_probabilities}).to_csv(csv_output_path, encoding='utf-8-sig', index=False)